In [1]:
import petsc4py
from petsc4py import PETSc
import underworld3 as uw
from underworld3.swarm import SwarmPICLayout

In [2]:
n_els = 3
mesh = uw.meshing.UnstructuredSimplexBox(
    minCoords=(0.0, 0.0), maxCoords=(1.0, 1.0), 
    cellSize=1 / n_els, qdegree=2, refinement=3
)


In [3]:
mesh.dm.view()

DM Object: uw_.meshes/uw_simplexbox_minC(0.0, 0.0)_maxC(1.0, 1.0)_csize0.3333333333333333_regFalse.msh 1 MPI process
  type: plex
uw_.meshes/uw_simplexbox_minC(0.0, 0.0)_maxC(1.0, 1.0)_csize0.3333333333333333_regFalse.msh in 2 dimensions:
  Number of 0-cells per rank: 881
  Number of 1-cells per rank: 2544
  Number of 2-cells per rank: 1664
Labels:
  celltype: 3 strata with value/size (1 (2544), 3 (1664), 0 (881))
  depth: 3 strata with value/size (0 (881), 1 (2544), 2 (1664))
  All_Boundaries: 1 strata with value/size (1001 (180))
  Bottom: 1 strata with value/size (11 (47))
  Elements: 1 strata with value/size (99999 (4402))
  Left: 1 strata with value/size (14 (47))
  Right: 1 strata with value/size (13 (47))
  Top: 1 strata with value/size (12 (47))
  Defined by transform from:
  DM_0x84000002_2 in 2 dimensions:
    Number of 0-cells per rank:   233  
    Number of 1-cells per rank:   648  
    Number of 2-cells per rank:   416  
  Labels:
    celltype: 3 strata with value/size (1 

In [6]:
v = uw.discretisation.MeshVariable("U", mesh, mesh.dim, degree=2)
p = uw.discretisation.MeshVariable("P", mesh, 1, degree=1)

In [7]:
swarm = uw.swarm.Swarm(mesh=mesh)
swarm.populate_petsc(
    fill_param=2,
    layout = SwarmPICLayout.GAUSS
)

In [8]:
# check the mesh if in a notebook / serial

if uw.mpi.size == 1:

    import numpy as np
    import pyvista as pv
    import vtk

    pv.global_theme.background = "white"
    pv.global_theme.window_size = [1250, 1250]
    pv.global_theme.anti_aliasing = "msaa"
    pv.global_theme.jupyter_backend = "panel"
    pv.global_theme.smooth_shading = True
    pv.global_theme.camera['viewup'] = [0.0, 1.0, 0.0]
    pv.global_theme.camera['position'] = [0.0, 0.0, 1.0]

    pvmesh = pv.read(".meshes/uw_simplexbox_minC(0.0, 0.0)_maxC(1.0, 1.0)_csize0.1_regFalse.msh")

    # with mesh.access():
    #     usol = v_soln.data.copy()

    # with mesh.access():
    #     pvmesh.point_data["Vmag"] = uw.function.evalf(
    #         sympy.sqrt(v_soln.sym.dot(v_soln.sym)), mesh.data
    #     )
    #     pvmesh.point_data["P"] = uw.function.evalf(p_soln.fn, mesh.data)


    # v_vectors = np.zeros((mesh.data.shape[0], 3))
    # v_vectors[:, 0] = uw.function.evalf(v_soln[0].sym, mesh.data)
    # v_vectors[:, 1] = uw.function.evalf(v_soln[1].sym, mesh.data)
    # pvmesh.point_data["V"] = v_vectors

    # arrow_loc = np.zeros((v_soln.coords.shape[0], 3))
    # arrow_loc[:, 0:2] = v_soln.coords[...]

    # arrow_length = np.zeros((v_soln.coords.shape[0], 3))
    # arrow_length[:, 0:2] = usol[...]

    # point sources at cell centres

    points = np.zeros((mesh._centroids.shape[0], 3))
    points[:, 0] = mesh._centroids[:, 0]
    points[:, 1] = mesh._centroids[:, 1]
    point_cloud = pv.PolyData(points)
    
    with swarm.access():
        spoints = np.zeros((swarm.particle_coordinates.data.shape[0], 3))
        spoints[:, 0] = swarm.particle_coordinates.data[:, 0]
        spoints[:, 1] = swarm.particle_coordinates.data[:, 1]
        spoint_cloud = pv.PolyData(spoints)


    # pvstream = pvmesh.streamlines_from_source(
    #     point_cloud, vectors="V", integration_direction="forward", max_steps=10
    # )

    pl = pv.Plotter()

    # pl.add_arrows(arrow_loc, arrow_length, mag=0.025 / U0, opacity=0.75)

    pl.add_points(spoint_cloud,color="Black",
                  render_points_as_spheres=False,
                  point_size=5, opacity=0.66
                )

    # pl.add_mesh(
    #     pvmesh,
    #     cmap="coolwarm",
    #     edge_color="Black",
    #     show_edges=True,
    #     scalars="Vmag",
    #     use_transparency=False,
    #     opacity=1.0,
    # )

    pl.add_mesh(pvmesh,'Black', 'wireframe', opacity=0.75)
    # pl.add_mesh(pvstream)

    # pl.remove_scalar_bar("mag")

    pl.show()